In [ ]:
#финальные задания модуля 3 pandas

import numpy as np
import pandas as pd
import re #библиотека регулярных выражений

# Задание 

data_1 = pd.DataFrame({'Value': [100, 45, 80],
                       'Group': [1, 4, 5]},
                      index = ['I0', 'I1', 'I2'])
data_2 = pd.DataFrame({'Company': ['Google', 'Amazon', 'Facebook'],
                       'Add': ['S0', 'S1', 'S7']},
                      index = ['I0', 'I1', 'I3']
                     )

#соединить по индексам, ведущая data_2
data_result = data_1.join(data_2, how='inner')
#display(data_result)

# Задание

a = pd.DataFrame({'A': ['a', 'b', 'c'], 'B': [103, 214, 124], 'C': [1, 4, 2]})
b = pd.DataFrame({'V': ['d', 'b', 'c'], 'U': [1393.7, 9382.2, 1904.5], 'C': [1, 3, 2]})
a.merge(b, how='right', on='C')


# Задание Сформируйте DataFrame merged, так чтобы после объединения purchase_df и items_df остались модели, 
# которые учтены на складе и имели продажи.

#инф по таблицам
# item_id — идентификатор модели;
# vendor — производитель модели;
# stock_count — имеющееся на складе количество данных моделей (в штуках);
# purchase_id — идентификатор покупки;
# price — стоимость модели в покупке.

items_df = pd.DataFrame({
            'item_id': [417283, 849734, 132223, 573943, 19475, 3294095, 382043, 302948, 100132, 312394],
            'vendor': ['Samsung', 'LG', 'Apple', 'Apple', 'LG', 'Apple', 'Samsung', 'Samsung', 'LG', 'ZTE'],
            'stock_count': [54, 33, 122, 18, 102, 43, 77, 143, 60, 19]
        })

purchase_df = pd.DataFrame({
            'purchase_id': [101, 101, 101, 112, 121, 145, 145, 145, 145, 221],
            'item_id': [417283, 849734, 132223, 573943, 19475, 3294095, 382043, 302948, 103845, 100132],
            'price': [13900, 5330, 38200, 49990, 9890, 33000, 67500, 34500, 89900, 11400]
        })

merged = items_df.merge(purchase_df, how='inner', on='item_id')
series = merged['price'] * merged['stock_count']
income = series.sum()

#19 729 490
#34 690 400

######################################################### *****Тоговая работа*****####################################
# Итоговое задание работа с таблице ratings_movies
# подгрузим таблицу
data_film = pd.read_csv('Data/ratings_movies.csv')
# display(data_film.head(n=2))
#   d:  0	userId	movieId	    rating	date	                title	                    genres
#   0	0	1	    1	        4.0	    2000-07-30 18:45:03	    Toy Story (1995)	        Adventure|Animation|Children|Comedy|Fantasy
#   1	1	1	    3	        4.0	    2000-07-30 18:20:47	    Grumpier Old Men (1995)	    Comedy|Romance
#функция выделения из названия фильма, год его выпуска

def get_year_release(arg):
    #находим все слова по шаблону "(DDDD)"
    candidates = re.findall(r'\(\d{4}\)', arg) 
    # проверяем число вхождений
    if len(candidates) > 0:
        #если число вхождений больше 0,
	#очищаем строку от знаков "(" и ")"
        year = candidates[0].replace('(', '')
        year = year.replace(')', '')
        return int(year)
    else:
        #если год не указан, возвращаем None
        return None
# 1 Задание Создайте в таблице новый признак year_release, который соответствует году выпуска фильма.
data_film ['year_release'] = data_film['title'].apply(get_year_release)
data_film['year_release'] = data_film['year_release'].fillna(0).astype(int)
#print(f' Всего фильмов без дат производства {data_film[data_film['year_release'] == 0].shape[0]}')

#****************************************************************************************************************************    
# 2 Задание Какой фильм, выпущенный в 1999 году, получил наименьшую среднюю оценку зрителей?
 
#получим таблицу с 4536 строками оценки фильмов
data_film_1999 = data_film[data_film['year_release'] == 1999]
#посмотрим сколько всего фильмов в рейтинге
data_film_1999['title'].nunique() #всего 261 фильм
# сгруппируем по названиям фильмов, и посчитаем средний рейтинг
data_film_1999.groupby('title')['rating'].mean().sort_values(ascending=True)
#****************************************************************************************************************************
# 3 Задание Какое сочетание жанров фильмов (genres), выпущенных в 2010 году, получило наименьшую среднюю оценку (rating)?
data_film_2010 = data_film[data_film['year_release'] == 2010]   
data_film_2010.groupby('genres')['rating'].mean().sort_values(ascending=True) 
#*****************************************************************************************************************************
# 4 Задание Какой пользователь (userId) посмотрел наибольшее количество различных (уникальных) комбинаций жанров (genres) фильмов? 
# В качестве ответа запишите идентификатор этого пользователя.
# сгруппируем по id, по столбцу genres найдем кол уникальных значений и отсортируем от макс к мин
data_film.groupby('userId')['genres'].nunique().sort_values(ascending=False)
#*****************************************************************************************************************************
# 5 Задание Найдите пользователя, который выставил наименьшее количество оценок, но его средняя оценка фильмам наибольшая.

# две маски, 1 = мин кол вхождений ID, rating.mean = max
# получим первую группировку по пользователям
data_film['userId'].nunique() #610 уникальных пользователей
#средний рейтинг от максимума к минимуму
s_1 = data_film.groupby('userId')['rating'].mean().sort_values(ascending = False)
#кол оценок от мин к макс
s_2 = data_film.groupby('userId')['rating'].count().sort_values(ascending = True)
result_user = pd.concat([s_1, s_2], axis=1, keys=['Mean rating', 'Number of rating'])
result_user.head(n=5)
#*******************************************************************************************************************************
# 6 Задание Найдите сочетание жанров (genres) за 2018 год, которое имеет наибольший средний рейтинг (среднее по столбцу rating), 
# и при этом число выставленных ему оценок (количество значений в столбце rating) больше 10.

# получим таблицу за 2018 год
data_film_2018 = data_film[data_film['year_release'] == 2018]
# средний рейтинг в группах жанра от макс к мин
s_1 = data_film_2018.groupby('genres')['rating'].mean().sort_values(ascending = False)
#кол оценок от мин к макс
s_2 = data_film_2018.groupby('genres')['rating'].count().sort_values(ascending = True)
result_user = pd.concat([s_1, s_2], axis=1, keys=['Mean rating', 'Number of rating'])
result_user.head(n=20)
#******************************************************************************************************************************
# задание 7 Добавьте в таблицу новый признак year_rating — год выставления оценки.
# Создайте сводную таблицу, которая иллюстрирует зависимость среднего рейтинга фильма от года выставления оценки и жанра. 
data_film['date'] = pd.to_datetime(data_film['date'])
data_film['year_rating'] = data_film['date'].dt.year
#теперь сгрупируем в сводную таблицу
pv_table = data_film.pivot_table(values='rating', index='genres', columns='year_rating', fill_value=0)

#******************************************************************************************************************************
# 8 задание прочитать таблицы и обьединить их по связующему признаку, найти макс выручку по ID покупателя, во внимание 
# взять только оплаченные ордеры

orders = pd.read_csv('Data/orders.csv', sep=';')
prod_data = pd.read_csv('Data/pd.csv', sep = ';')
#обьединим таблицы по индетификаторам товара, за основу возьмем левую таб, там инф о продажах
result_prod = orders.merge(prod_data, how='left', right_on='Product_ID', left_on='ID товара')
#в итоговой таблице удаляем дубль индентификатора товара
result_prod = result_prod.drop('Product_ID', axis=1)
display(result_prod)
#сделаем выборку по оплаченым заказам
order_end = result_prod[result_prod['Оплачен']=='Да']
#почистим от NaN
order_end['Price'] = order_end['Price'].fillna(0).astype(int)
#введем новый столбец общая сумма по ордеру
order_end['TotalSale'] = order_end['Количество'] * order_end['Price']
#сгруппируем по покупателям и отсортируем по общей выручки
itog = order_end.groupby('ID Покупателя')['TotalSale'].sum().sort_values(ascending=False)
display(itog)







,Дата создания,Order ID,ID Покупателя,Статус,Оплачен,Отменен,Отгружен,ID товара,Количество,Name,Price,CURRENCY
0,09.11.2019 21:55:51,9,10,"Принят, ожидается оплата",Нет,Нет,Нет,103,5,"Носки Подарочные, муж",199.0,RUR
1,09.11.2019 15:05:57,8,9,"Принят, ожидается оплата",Нет,Нет,Нет,86,100,"Носки Простые, муж",45.0,RUR
2,09.11.2019 15:05:57,8,9,"Принят, ожидается оплата",Нет,Нет,Нет,104,10,"Носки Подарочные, жен",249.0,RUR
3,09.11.2019 12:50:07,7,8,"Принят, ожидается оплата",Нет,Нет,Нет,104,7,"Носки Подарочные, жен",249.0,RUR
4,09.11.2019 12:00:00,6,1,"Принят, ожидается оплата",Нет,Нет,Нет,104,5,"Носки Подарочные, жен",249.0,RUR
5,09.11.2019 12:00:00,6,1,"Принят, ожидается оплата",Нет,Нет,Нет,103,5,"Носки Подарочные, муж",199.0,RUR
6,08.11.2019 08:36:22,5,5,Отменён,Нет,Да,Нет,124,1,Носки беговые Camino,999.0,RUR
7,08.11.2019 08:36:22,4,9,"Принят, ожидается оплата",Нет,Нет,Да,91,1,"Носки Честные, муж",50.0,RUR
8,08.11.2019 08:36:22,3,8,"Оплачен, формируется к отправке",Да,Нет,Нет,103,3,"Носки Подарочные, муж",199.0,RUR
9,08.11.2019 08:36:22,3,8,"Оплачен, формируется к отправке",Да,Нет,Нет,104,3,"Носки Подарочные, жен",249.0,RUR


C:\Users\Art\AppData\Local\Temp\ipykernel_14556\2949128037.py:148: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  order_end['Price'] = order_end['Price'].fillna(0).astype(int)
C:\Users\Art\AppData\Local\Temp\ipykernel_14556\2949128037.py:150: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  order_end['TotalSale'] = order_end['Количество'] * order_end['Price']


ID Покупателя
7    17096
5    13043
8     1344
1        0
Name: TotalSale, dtype: int64